In [49]:
import torch
from torch import nn
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch.optim as optim

In [50]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [51]:
# Load data (one large matrix?)
datafiles = ['../project/dataset1.csv', '../project/dataset2.csv', '../project/dataset3.csv', '../project/dataset4.csv', '../project/dataset5.csv']
n_samples = 50000           # Number of samples
n_dataset = len(datafiles)  # Number of datasets
train = 0.9   # 245000 (77.777...%)
test = 0.1    # 35000 (10%)
# val = 1-train # 70000 (Rest)

size_test = int(n_samples*n_dataset*test)+1
size_train = int((n_samples*n_dataset-size_test)*train)+1
# size_val = int((n_samples*n_dataset-size_test)*val)+1

In [52]:
# Split all datasets
datasets = np.empty((int(n_samples*n_dataset),5))
print(datasets.shape)
for i,datafile in enumerate(datafiles):
  dataset = np.loadtxt(datafile, delimiter=',')
  datasets[int(i*n_samples):int(n_samples*(i+1)),:] = dataset
print(datasets[0,0])
# Split into input and output
Cr = datasets[:,0:4]   # Input: relative entropy of coherence
ES = np.array([float(y) for y in datasets[:,4]])  # Output: entangled/separable

# Count number of entangled and separable states
n_entangled = len([es for es in ES if es == 1])
print(f'Number of entangled states: {n_entangled}\n')
print(f'Number of separable states: {n_samples*n_dataset-n_entangled}\n')
# Split into train-and-validation and test
Cr_trainval, Cr_test, ES_trainval, ES_test = train_test_split(Cr, ES, test_size=test,
                                                      random_state=42)
# # Split into train and validation
# Cr_train, Cr_val, ES_train, ES_val = train_test_split(Cr_trainval, ES_trainval, test_size=val,
#                                                       random_state=30)
# DEBUG
# print(np.shape(Cr_train))
# print(np.shape(Cr_test))
# print(np.shape(Cr_val))


(250000, 5)
0.195132062642601
Number of entangled states: 127330

Number of separable states: 122670



In [53]:
# Define the neural network architecture
class LinearRegressionWithHidden(nn.Module):
    def __init__(self):
        super(LinearRegressionWithHidden, self).__init__()
        self.layer1= nn.Linear(4, 5)
        self.layer2= nn.Linear(5, 1)
        self.output= nn.Linear(1, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.apply(self._init_weights)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.layer2(x)
        output = self.sigmoid(self.output(x))
        output = torch.where(output <= 4, torch.tensor(0.0), torch.tensor(1.0))
        output.requires_grad = True
        return output

    def _init_weights(self, l):
      # Initialize weights uniformly
      if isinstance(l, nn.Linear):
        nn.init.kaiming_uniform_(l.weight)
        #nn.init.uniform_(l.weight)
        l.bias.data.fill_(0.01)

# # Instantiate the model
# model = LinearRegressionWithHidden()

# # Define loss function and optimizer
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# # Convert numpy arrays to PyTorch tensors
# X_train_tensor = torch.tensor(Cr_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(ES_train, dtype=torch.float32)

# # Train the model
# num_epochs =
# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     outputs = model(X_train_tensor)
#     loss = criterion(outputs, y_train_tensor)
#     loss.backward()
#     optimizer.step()

#     if (epoch+1) % 10 == 0:
#         print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# # Evaluate the model
# with torch.no_grad():
#     predicted = model(X_train_tensor).detach().numpy()
#     print("Training Loss:", criterion(torch.tensor(predicted), y_train_tensor).item())

# # Get the learned parameters
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, param.data)

In [54]:

model = LinearRegressionWithHidden()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Split data into train and test sets
Cr_trainval, Cr_test, ES_trainval, ES_test = train_test_split(Cr, ES, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
Cr_trainval_tensor = torch.tensor(Cr_trainval, dtype=torch.float32)
ES_trainval_tensor = torch.tensor(ES_trainval, dtype=torch.float32)
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(Cr_trainval_tensor)
    ES_trainval_tensor = ES_trainval_tensor.view(-1, 1)

    # Calculate the loss
    loss = criterion(outputs, ES_trainval_tensor)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# After training, you can evaluate the model on the test set
with torch.no_grad():
    Cr_test_tensor = torch.tensor(Cr_test, dtype=torch.float32)
    ES_test_tensor = torch.tensor(ES_test, dtype=torch.float32)

    # Forward pass through the model to get predictions
    outputs = model(Cr_test_tensor)

    # Convert the output tensor to have the same shape as the target tensor
    outputs = outputs.view(-1)  # Ensure the output tensor is 1D

    # Calculate the test loss
    test_loss = criterion(outputs, ES_test_tensor)

    print(f'Test Loss: {test_loss.item():.4f}')

Epoch [10/100], Loss: 0.5090
Epoch [20/100], Loss: 0.5090
Epoch [30/100], Loss: 0.5090
Epoch [40/100], Loss: 0.5090
Epoch [50/100], Loss: 0.5090
Epoch [60/100], Loss: 0.5090
Epoch [70/100], Loss: 0.5090
Epoch [80/100], Loss: 0.5090
Epoch [90/100], Loss: 0.5090
Epoch [100/100], Loss: 0.5090
Test Loss: 0.5106
